In [4]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.applications import MobileNetV2

In [11]:
def buid_object_detection_model(input_shape=(224,224,3), num_classes = 10):
    backbone = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
    backbone.trainable = False
    
    model = models.Sequential()
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(num_classes+4, activation='linear'))
    
    return model

model = buid_object_detection_model()
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [12]:
classification_loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [13]:
def bbox_loss(y_true, y_pred):
    return tf.reduce_mean(tf.keras.mean_squared_error(y_true, y_pred))

In [17]:
def custom_loss(y_true, y_pred):
    class_pred, bbox_pred = y_pred[:,:-4], y_pred[:, -4:]
    class_true, bbox_true = y_true[:,:-4], y_true[:, -4:]
    
    cls_loss = classification_loss(class_true, class_pred)
    box_loss = bbox_loss(bbox_true, bbox_pred)
    
    return cls_loss + box_loss

model.compile(optimizer = 'adam', loss = custom_loss)

In [ ]:
model.fit(train_data, epochs=10, validation_data= val_data)